# Análise Exploratória de Dados - Hackathon Forecast 2025

Este notebook contém uma análise exploratória completa dos dados de vendas com visualizações interativas para identificar padrões, sazonalidade e insights para o modelo de previsão.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pyo.init_notebook_mode(connected=True)

# Importar módulos do projeto
import sys
sys.path.append('../src')
from data.ingestion import DataIngestion
from data.preprocessing import DataPreprocessor
from utils.eda_utils import EDAUtils

## 1. Carregamento e Preparação dos Dados

In [ ]:
# Carregar dados
ingestion = DataIngestion()
preprocessor = DataPreprocessor()
eda_utils = EDAUtils()

# Carregar dados dos arquivos Parquet
print("Carregando dados...")
transactions_df = ingestion.load_transactions('../hackathon_2025_templates/')
products_df = ingestion.load_products('../hackathon_2025_templates/')
stores_df = ingestion.load_stores('../hackathon_2025_templates/')

print(f"Transações: {transactions_df.shape}")
print(f"Produtos: {products_df.shape}")
print(f"PDVs: {stores_df.shape}")

In [ ]:
# Pré-processamento básico
print("Processando dados...")
transactions_clean = preprocessor.clean_transactions(transactions_df)
weekly_sales = preprocessor.aggregate_weekly_sales(transactions_clean)
merged_data = preprocessor.merge_master_data(weekly_sales, products_df, stores_df)

print(f"Dados processados: {merged_data.shape}")
print(f"Período: {merged_data['data'].min()} a {merged_data['data'].max()}")

## 2. Visão Geral dos Dados

In [ ]:
# Estatísticas descritivas interativas
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição de Vendas', 'Vendas por Categoria', 
                   'Vendas por Tipo de PDV', 'Evolução Temporal'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Distribuição de vendas
fig.add_trace(
    go.Histogram(x=merged_data['quantidade'], name='Quantidade', nbinsx=50),
    row=1, col=1
)

# Vendas por categoria
cat_sales = merged_data.groupby('categoria_produto')['quantidade'].sum().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=cat_sales.index, y=cat_sales.values, name='Por Categoria'),
    row=1, col=2
)

# Vendas por tipo de PDV
pdv_sales = merged_data.groupby('tipo_pdv')['quantidade'].sum()
fig.add_trace(
    go.Bar(x=pdv_sales.index, y=pdv_sales.values, name='Por Tipo PDV'),
    row=2, col=1
)

# Evolução temporal
temporal_sales = merged_data.groupby('data')['quantidade'].sum()
fig.add_trace(
    go.Scatter(x=temporal_sales.index, y=temporal_sales.values, 
              mode='lines', name='Vendas Totais'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Visão Geral dos Dados de Vendas")
fig.show()

## 3. Análise de Sazonalidade

In [ ]:
# Análise de sazonalidade por semana do ano
merged_data['semana_ano'] = merged_data['data'].dt.isocalendar().week
merged_data['mes'] = merged_data['data'].dt.month
merged_data['dia_semana'] = merged_data['data'].dt.dayofweek

# Sazonalidade semanal
weekly_pattern = merged_data.groupby('semana_ano')['quantidade'].agg(['mean', 'std']).reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=weekly_pattern['semana_ano'],
    y=weekly_pattern['mean'],
    mode='lines+markers',
    name='Média Semanal',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=weekly_pattern['semana_ano'],
    y=weekly_pattern['mean'] + weekly_pattern['std'],
    mode='lines',
    line=dict(color='lightblue', width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=weekly_pattern['semana_ano'],
    y=weekly_pattern['mean'] - weekly_pattern['std'],
    mode='lines',
    line=dict(color='lightblue', width=0),
    fill='tonexty',
    name='±1 Desvio Padrão'
))

fig.update_layout(
    title='Padrão Sazonal por Semana do Ano',
    xaxis_title='Semana do Ano',
    yaxis_title='Quantidade Média',
    height=500
)
fig.show()

In [ ]:
# Heatmap de vendas por mês e categoria
monthly_category = merged_data.groupby(['mes', 'categoria_produto'])['quantidade'].sum().unstack(fill_value=0)

fig = px.imshow(
    monthly_category.T,
    labels=dict(x="Mês", y="Categoria", color="Quantidade"),
    title="Heatmap: Vendas por Mês e Categoria",
    aspect="auto"
)
fig.show()

## 4. Análise por PDV e Produtos

In [ ]:
# Top PDVs por volume de vendas
top_pdvs = merged_data.groupby('pdv')['quantidade'].sum().sort_values(ascending=False).head(20)

fig = px.bar(
    x=top_pdvs.index.astype(str),
    y=top_pdvs.values,
    title="Top 20 PDVs por Volume de Vendas",
    labels={'x': 'PDV', 'y': 'Quantidade Total'}
)
fig.update_xaxis(tickangle=45)
fig.show()

In [ ]:
# Análise de produtos mais vendidos
top_products = merged_data.groupby('produto')['quantidade'].sum().sort_values(ascending=False).head(20)

fig = px.bar(
    x=top_products.index.astype(str),
    y=top_products.values,
    title="Top 20 Produtos por Volume de Vendas",
    labels={'x': 'Produto', 'y': 'Quantidade Total'}
)
fig.update_xaxis(tickangle=45)
fig.show()

## 5. Análise de Distribuições e Outliers

In [ ]:
# Box plots interativos por categoria
fig = px.box(
    merged_data,
    x='categoria_produto',
    y='quantidade',
    title="Distribuição de Vendas por Categoria (com Outliers)",
    points="outliers"
)
fig.update_xaxis(tickangle=45)
fig.show()

In [ ]:
# Análise de outliers por tipo de PDV
fig = px.violin(
    merged_data,
    x='tipo_pdv',
    y='quantidade',
    box=True,
    title="Distribuição de Vendas por Tipo de PDV"
)
fig.show()

## 6. Análise de Correlações

In [ ]:
# Preparar dados numéricos para correlação
numeric_cols = ['quantidade', 'semana_ano', 'mes', 'dia_semana']
correlation_data = merged_data[numeric_cols].corr()

fig = px.imshow(
    correlation_data,
    text_auto=True,
    aspect="auto",
    title="Matriz de Correlação - Variáveis Numéricas"
)
fig.show()

## 7. Análise Temporal Detalhada

In [ ]:
# Série temporal com decomposição
daily_sales = merged_data.groupby('data')['quantidade'].sum().reset_index()

fig = px.line(
    daily_sales,
    x='data',
    y='quantidade',
    title="Evolução Temporal das Vendas - 2022"
)

# Adicionar média móvel
daily_sales['ma_7'] = daily_sales['quantidade'].rolling(window=7).mean()
daily_sales['ma_30'] = daily_sales['quantidade'].rolling(window=30).mean()

fig.add_trace(go.Scatter(
    x=daily_sales['data'],
    y=daily_sales['ma_7'],
    mode='lines',
    name='Média Móvel 7 dias',
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=daily_sales['data'],
    y=daily_sales['ma_30'],
    mode='lines',
    name='Média Móvel 30 dias',
    line=dict(color='red')
))

fig.update_layout(height=500)
fig.show()

## 8. Insights e Conclusões

In [ ]:
# Resumo estatístico
print("=== RESUMO DA ANÁLISE EXPLORATÓRIA ===")
print(f"\nPeríodo analisado: {merged_data['data'].min()} a {merged_data['data'].max()}")
print(f"Total de registros: {len(merged_data):,}")
print(f"Número de PDVs únicos: {merged_data['pdv'].nunique():,}")
print(f"Número de produtos únicos: {merged_data['produto'].nunique():,}")
print(f"Número de categorias: {merged_data['categoria_produto'].nunique()}")

print("\n=== ESTATÍSTICAS DE VENDAS ===")
print(f"Vendas totais: {merged_data['quantidade'].sum():,}")
print(f"Média diária: {merged_data['quantidade'].mean():.2f}")
print(f"Mediana: {merged_data['quantidade'].median():.2f}")
print(f"Desvio padrão: {merged_data['quantidade'].std():.2f}")

print("\n=== TOP CATEGORIAS ===")
top_categories = merged_data.groupby('categoria_produto')['quantidade'].sum().sort_values(ascending=False)
for i, (cat, sales) in enumerate(top_categories.head(5).items()):
    print(f"{i+1}. {cat}: {sales:,} unidades")

print("\n=== INSIGHTS PRINCIPAIS ===")
print("1. Identificar padrões sazonais claros")
print("2. Variação significativa entre PDVs")
print("3. Concentração em algumas categorias")
print("4. Presença de outliers que precisam ser tratados")
print("5. Tendências temporais que podem ser modeladas")